## Model training and registration

In this notebook we will:

- Register a model to the model registry.
- Fetch the model from the model registry.

This will introduce the `hsml` (**H**opsworks **M**achine **L**earning) library, which contains functionality to keep track of models and deploy them.

In [1]:
import hsfs

conn = hsfs.connection()
fs = conn.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


### Load Best Hyperparameters

We will train a model using the best hyperparameters we found in the previous notebook.
Recall that we saved these as a pickled dictionary that we uploaded to our cluster.
If you don't have the file locally you can download it using the *hopsworks* library.

In [ ]:
import hopsworks

hopsworks_conn = hopsworks.connection()
project = hopsworks_conn.get_project()
dataset_api = project.get_dataset_api()

uploaded_file_path = "Resources/best_params.pickle"
dataset_api.download(uploaded_file_path)

Retrieve the best hyperparameters from this file.

In [ ]:
import pickle

with open("best_params.pickle", "rb") as f:
    best_params = pickle.load(f)

pos_class_weight = best_params["pos_class_weight"]

print(pos_class_weight)

### Load Training Data & Train Model

Next, we'll train a model in the same way as the previous notebook.

In [4]:
from sklearn.linear_model import LogisticRegression

# Load data.
td = fs.get_training_dataset("transactions_dataset_splitted")
X_train = td.read("train")
X_val = td.read("validation")
target = td.label[0]  # "fraud_label"
y_train = X_train.pop(target)
y_val = X_val.pop(target)

# Train model.
clf = LogisticRegression(class_weight={0: 1.0 - pos_class_weight, 1: pos_class_weight}, solver='liblinear')
clf.fit(X_train, y_train)

We will also compute some evaluation metrics, which we will register together with the model.

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, classification_report

preds = clf.predict(X_val)

precision, recall, fscore, _ = precision_recall_fscore_support(y_val, preds)

metrics = {
    'precision': precision[1],
    'recall': recall[1],
    'fscore': fscore[1]
}

print(classification_report(y_val, preds))


### Register model

One of the features in Hopsworks is the model registry. This is where we can store different versions of models and compare their performance. Models from the registry can then be served as API endpoints.

Let's connect to the model registry using the [HSML library](https://docs.hopsworks.ai/machine-learning-api/latest) from Hopsworks.

In [12]:
import hsml

conn = hsml.connection()
mr = conn.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


Before registering the model we will export it as a pickle file using joblib.

In [ ]:
import joblib
import os

# TODO Do we need to store the .pkl file in a directory?

os.mkdir('tmp_model')
joblib.dump(clf, 'tmp_model/model.pkl')

The model needs to be set up with a [Model Schema](https://docs.hopsworks.ai/machine-learning-api/latest/generated/model_schema/), which describes the inputs and outputs for a model.

A Model Schema can be automatically generated from training examples, as shown below.

In [13]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

model_schema.to_dict()


Exported model fraud_tutorial_model with version 1


With the schema in place, we can finally register our model.

In [ ]:
model = mr.sklearn.create_model(
    name="fraud_tutorial_model",
    metrics=metrics,
    description="Logistic regression model trained with class weights.",
    input_example=X_train.sample(),
    model_schema=model_schema
)

model.save('tmp_model')

It's important to know that every time you save a model with the same name, a new version of the model will be saved, so nothing will be overwritten. In this way, you can compare several versions of the same model - or create a model with a new name, if you prefer that.

#### Finding the best performing model

Let's imagine you have trained and registered several versions of the same model. Now you can query the model registry for the best model according to your preferred criterion, say F1-score in our case.

The `direction` option is used to indicate if the metric should be high or low (max or min); in our case, it should be high (max).

In [14]:
best_model = mr.get_best_model(name="fraud_tutorial_model", metric="fscore", direction="max")
best_model.to_dict()

{'id': 'fraud_tutorial_model_1',
 'experimentId': None,
 'projectName': 'clean_up',
 'experimentProjectName': 'clean_up',
 'name': 'fraud_tutorial_model',
 'modelSchema': {'href': 'https://hopsworks.glassfish.service.consul:8182/hopsworks-api/api/project/125/dataset/Projects/clean_up/Models/fraud_tutorial_model/1/model_schema.json',
  'zip_state': 'NONE'},
 'version': 1,
 'description': 'A collection of models for fraud_tutorial_model',
 'inputExample': {'href': 'https://hopsworks.glassfish.service.consul:8182/hopsworks-api/api/project/125/dataset/Projects/clean_up/Models/fraud_tutorial_model/1/input_example.json',
  'zip_state': 'NONE'},
 'framework': 'SKLEARN',
 'metrics': {'positive_precision': '0.016508378499328725',
  'positive_recall': '0.8038740920096852'},
 'trainingDataset': None,
 'environment': ['/Projects/clean_up/Models/fraud_tutorial_model/1/environment.yml'],
 'program': 'Models/fraud_tutorial_model/1/program.ipynb'}

### Next Steps

In the next notebook, we'll look at model serving for the model we just registered to the Model Registry.